### Decorators

Let's start with the pattern we have for creating decorators - it is pretty much the same, so you can re-use this pattern with any customizations you want:

In [1]:
def wrapper(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return inner

You'll notice that `inner` will call `func` with `*args` and `**kwargs` and nothing else.

It may not look very interesting, but notice that we can now wrap any function:

In [2]:
def add(a, b, c):
    return a + b + c

def greet(name):
    return f'Hello {name}!'

def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join(
        [
            item_sep.join(str(item) for item in row) 
            for row in data
        ]
    )     

We can call those functions as they are:

In [3]:
add(1, 2, 3)

6

In [4]:
greet('Python')

'Hello Python!'

In [5]:
join([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

'1,2,3\n4,5,6\n7,8,9'

But we can also create these wrapped functions:

In [6]:
add_wrapped = wrapper(add)
greet_wrapped = wrapper(greet)
join_wrapped = wrapper(join)

And we can call these "wrapped" functions just like we called the "non-wrapped" original versions:

In [7]:
add_wrapped(1, 2, 3)

6

In [8]:
greet_wrapped('Python')

'Hello Python!'

In [9]:
join_wrapped([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

'1,2,3\n4,5,6\n7,8,9'

So, as it stands, that `wrapper` function does not do much - but it forms the basis for us to add functionality around our original function however we want.

Let's say we want a simple log of the call being made:

In [10]:
def log(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} called... result={result}')
        return result
    return inner

In [11]:
add_logged = log(add)
greet_logged = log(greet)
join_logged = log(join)

In [12]:
add_logged(1, 2, 3)

add called... result=6


6

In [13]:
greet_logged('Python')

greet called... result=Hello Python!


'Hello Python!'

So, we now have to remember to call `greet_logged` instead of `greet` everywhere in our code (assuming we want to log things everywhere).

Firstly, I really don't want to write code like `greet_logged`, `add_logged`, etc - I really just want to use `greet`, `add`, etc.

Secondly, I may already have thousands of lines of code that call `greet` `add`, etc - again, I really do not want to look for those calls and change each one to the `_logged` version.

So, instead I'm going to name the wrapped version the same as the original version. The wrapped version is a closure that still maintains a link to the original function, but my symbol `add`, `greet`, etc now points to the new function (closure).

In [14]:
def log(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} called... result={result}')
        return result
    return inner

def add(a, b, c):
    return a + b + c
add = log(add)

def greet(name):
    return f'Hello {name}!'
greet = log(greet)

def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join([item_sep.join(str(item) for item in row) for row in data])     
join = log(join)

And now I can use the same symbol to call the logged version:

In [15]:
greet('Python')

greet called... result=Hello Python!


'Hello Python!'

This pattern of defining the function `add` and then redefining the symbol using some decorator: `add = log(add)` is so common that there is a shorthand syntax for it:

In [16]:
def log(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} called... result={result}')
        return result
    return inner

@log
def add(a, b, c):
    return a + b + c

@log
def greet(name):
    return f'Hello {name}!'

@log
def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join([item_sep.join(str(item) for item in row) for row in data])     

In [17]:
add(1, 2, 3)

add called... result=6


6

Now every time we call one of those decorated functions in our code, a log will be emitted.

And this means we now have a central, unique location where we can change what our log decorator does - and every function that is decorated with the `log` decorator will use that definition - no re-typing code in multiple places.

In fact, let's now write proper logs,using Python's logging system (we'll just log to the console, but it can be set up to log to file, and a variety of other places).

In [18]:
import logging

We'll configure our logger (we only need to do this once per application):

In [19]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.DEBUG
)

In [20]:
logger = logging.getLogger('Custom Log')

In [21]:
logger.debug('Information message')

2021-01-17 09:30:21,507 DEBUG: Information message


In [22]:
logger.error('Some error happened')

2021-01-17 09:30:21,511 ERROR: Some error happened


In [23]:
logger.warning('Some warning')

2021-01-17 09:30:21,517 WARNING: Some warning


let's use that to write a better logging decorator, that will include the run time of the function as well:

In [24]:
from time import perf_counter

def log(func):
    def inner(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()
        logger.debug(f'called={func.__name__}, elapsed={end-start}')
        return result
    return inner

In [25]:
@log
def add(a, b, c):
    return a + b + c

@log
def greet(name):
    return f'Hello {name}!'

@log
def join(data, *, item_sep=',', line_sep='\n'):
    return line_sep.join([item_sep.join(str(item) for item in row) for row in data])     

In [26]:
add(10, 20, 30)

2021-01-17 09:30:21,536 DEBUG: called=add, elapsed=2.3590000000073275e-06


60

In [27]:
join([range(10) for _ in range(10)])

2021-01-17 09:30:21,546 DEBUG: called=join, elapsed=3.6437000000000275e-05


'0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9\n0,1,2,3,4,5,6,7,8,9'

We'll see other applications of decorators in the next set of videos.

The main takeaway is that decorators are very handy for adding pre and post function call code that is reusable across multiple functions - in a way that is completely **transparent** to the user.

For example, you may inherit some piece of code that defines several functions, and calls them hundreds of times - you can now log each of these functions by simply decorating the function definition - all other calls will transparently use the decorated function witout further modifications.

More often than not, you will probably end up using decorators that someone *else* has written in some library - but it is important to understand how they work if you want to use those effectively (and more importantly, understand what's going on!).